In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import models, layers, optimizers, callbacks
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.core import Flatten, Dense, Dropout
from keras import models, layers, optimizers
from keras import backend as K

K.clear_session()
# Loading datasets
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print('Train images shape:', train_images.shape)
print('Train labels shape:', train_labels.shape, '\n')
print('Test images shape:', test_images.shape)
print('Test labels shape:', test_labels.shape, '\n')

# Showing the First MNIST handwritten number
# plt.imshow(train_images[1], cmap='Greys')
# plt.show()
# print('Train image label shown:', train_labels[1], '\n')

# Training, Validation and Test datasets
valid_images = train_images[50000:60000]
valid_labels = train_labels[50000:60000]
train_images = train_images[0:50000]  # test images remain untouched
train_labels = train_labels[0:50000]

# Creating Tensors
train_images = train_images.reshape((50000, 28, 28, 1))
train_images = train_images.astype('float32') / 255  # converting to a [0, 1] scale
valid_images = valid_images.reshape((10000, 28, 28, 1))
valid_images = valid_images.astype('float32') / 255  # converting to a [0, 1] scale
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255  # converting to a [0, 1] scale

# One-hot encode labels
print('A label:', train_labels[19])
train_labels = to_categorical(train_labels)
valid_labels = to_categorical(valid_labels)
test_labels = to_categorical(test_labels)
print('A one-hot encode label:', train_labels[19])

In [1]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense
from keras.models import Model
import numpy as np

#Get back the convolutional part of a VGG network trained on ImageNet
vgg16_conv = VGG16(weights='imagenet', include_top=False)

# Freeze the layers except the last 4 layers
for layer in vgg16_conv.layers[:-4]:
    layer.trainable = False
    
# Check the trainable status of the individual layers
for layer in vgg16_conv.layers:
    print(layer, layer.trainable)
    
#Create your own input format (here 3x200x200)
input = Input(shape=(28,28,3),name = 'image_input')

#Use the generated model 
output_vgg16_conv = vgg16_conv(input)

#Add the fully-connected layers 
x = Dense(4096, activation='relu')(output_vgg16_conv)
x = Dropout(0.5)(x)
x = Dense(10, activation='softmax')(x)

#Create your own model 
my_model = Model(input=input, output=x)

#In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
my_model.summary()

compile = my_model.compile(optimizer=optimizers.RMSprop(lr=1e-5), loss='binary_crossentropy',
                           metrics=['accuracy'])

# Settting Callbacks
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.95, patience=3,verbose=1, cooldown=2)
csv_logger = CSVLogger(fold+'/log.csv', append=True, separator=';')

callb_l = [reduce_lr, csv_logger]
# Train the model
my_model.f# CNN Training
fit = my_model.fit(x=train_images, y=train_labels, steps_per_epoch=20, epochs=50,validation_data=(valid_images, valid_labels))
# Save Model
my_model.save(filepath=r'media\avell\model_save.h5', overwrite=True)

Using TensorFlow backend.


(<keras.engine.topology.InputLayer object at 0x7f08973d0890>, False)
(<keras.layers.convolutional.Conv2D object at 0x7f08973d0f90>, False)
(<keras.layers.convolutional.Conv2D object at 0x7f08973d0f50>, False)
(<keras.layers.pooling.MaxPooling2D object at 0x7f08973ef250>, False)
(<keras.layers.convolutional.Conv2D object at 0x7f08973ef850>, False)
(<keras.layers.convolutional.Conv2D object at 0x7f0896b95cd0>, False)
(<keras.layers.pooling.MaxPooling2D object at 0x7f0896b3ef50>, False)
(<keras.layers.convolutional.Conv2D object at 0x7f0896b4de10>, False)
(<keras.layers.convolutional.Conv2D object at 0x7f0896b5ed90>, False)
(<keras.layers.convolutional.Conv2D object at 0x7f0896afdf90>, False)
(<keras.layers.pooling.MaxPooling2D object at 0x7f0896b0ca50>, False)
(<keras.layers.convolutional.Conv2D object at 0x7f0896b28f50>, False)
(<keras.layers.convolutional.Conv2D object at 0x7f0896b3ad50>, False)
(<keras.layers.convolutional.Conv2D object at 0x7f0896ad5f10>, False)
(<keras.layers.poolin

ValueError: Negative dimension size caused by subtracting 2 from 1 for 'vgg16/block5_pool/MaxPool' (op: 'MaxPool') with input shapes: [?,1,1,512].

In [ ]:
# CNN Architecture
my_model = models.Sequential()
my_model.add(layers.Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1),
                           use_bias=True, input_shape=(28, 28, 1)))
my_model.add(layers.Activation('relu'))
my_model.add(layers.Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1), use_bias=True))
my_model.add(layers.Activation('relu'))

my_model.add(layers.MaxPooling2D(pool_size=(2, 2)))

my_model.add(layers.Dropout(rate=0.2))

my_model.add(layers.Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1), use_bias=True))
my_model.add(layers.Activation('relu'))

my_model.add(layers.MaxPooling2D(pool_size=(2, 2)))

my_model.add(layers.Flatten())

my_model.add(layers.Dropout(rate=0.2))

my_model.add(layers.Dense(units=10, use_bias=True))
my_model.add(layers.Activation('relu'))

my_model.add(layers.Dense(units=10, use_bias=True))
my_model.add(layers.Activation('softmax'))
my_model.summary()

#CNN Loss and Optimizer
compile = my_model.compile(optimizers.sgd(lr=0.3, decay=0.001), loss='categorical_crossentropy',
                           metrics=['accuracy'])

# CNN Training
fit = my_model.fit(x=train_images, y=train_labels, steps_per_epoch=20, epochs=50,validation_data=(valid_images, valid_labels))

# Save Model
my_model.save(filepath=r'media\avell\model_save.h5', overwrite=True)

In [ ]:
# CNN Architecturem for VGG16
model = models.Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(28, 28, 1)))
model.add(Conv2D(64, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(64, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(128, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(128, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

# model.add(ZeroPadding2D((1,1)))
# model.add(Conv2D(512, 3, 3, activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Conv2D(512, 3, 3, activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Conv2D(512, 3, 3, activation='relu'))
# model.add(MaxPooling2D((2,2), strides=(2,2)))

# model.add(ZeroPadding2D((1,1)))
# model.add(Conv2D(512, 3, 3, activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Conv2D(512, 3, 3, activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Conv2D(512, 3, 3, activation='relu'))
# model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

print(model.summary())

compile = model.compile(optimizers.sgd(lr=0.3, decay=0.01), loss='categorical_crossentropy',metrics=['accuracy'])
fit = model.fit(x=train_images, y=train_labels, batch_size=2500, epochs=100,validation_data=(valid_images, valid_labels))


# Saving Model
model.save(filepath=r'teste_cnn.h5', overwrite=True)